## Installation

In [1]:
%pip install -q pyomo
!apt-get install coinor-cbc

Note: you may need to restart the kernel to use updated packages.
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?


#### Initializing libraries and model

In [ ]:
from pyomo.environ import *
import math
model=ConcreteModel()
model.constraints=ConstraintList()

Add initial variables here, like importing the game files, the rules, etc, which are constants

#### Instruction for modelling
Add variables by ``model.x=Var(range(_dimension_),domain=[_the domain of the variable_])``

Add constraints by ``model.constraint.add(expr=...)``

## Solver using CBC

In [ ]:
opt = SolverFactory('cbc')
result=opt.solve(model,tee=True)
print('Solver status: ',result.solver.status)
print('Solver termination condition: ',result.solver.termination_condition)

In [ ]:
print('Optimal N',model.obj())